In [ ]:
# import matplotlib.pyplot as plt
import pathlib
import mne
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
from mne.channels.montage import get_builtin_montages
from mne.io import read_raw_bdf, concatenate_raws
import os
import numpy as np
from numpy import savetxt
from numpy import loadtxt

import matplotlib
matplotlib.use('Qt5Agg')

In [ ]:
# define path:
data_folder = '/Users/christinadelta/Desktop/os_data/economic/sub-10'

raw_file = os.path.join(data_folder, 'sub_10_economic_block_01.bdf')
raw = read_raw_bdf(raw_file,eog=['EXG3', 'EXG4','EXG5', 'EXG6'], stim_channel='auto',
                   exclude=(['EXG7', 'EXG8']), preload=True)

In [ ]:
raw.plot()

In [ ]:
events = mne.find_events(raw)

In [ ]:

event_id = {
    "sample": 1,
    "feed_1": 13,
    "feed_2": 14,
    "resprompt": 15,
    "start": 100,
    "end": 101
}

In [ ]:

event_id = {
    "sample": 1,
    "feed_1": 13,
    "resprompt": 15,
    "start": 100,
    "end": 101
}

In [ ]:
filtered = raw.copy().filter(l_freq=1, h_freq=20) 

filtered.plot(title='filtered')
#filtered2.plot(title='filtered2')

In [ ]:
# re-reference
ref_channels = ['EXG1', 'EXG2']
filtered.set_eeg_reference(ref_channels)

In [ ]:
filtered.plot()

In [ ]:
biosemi_montage = mne.channels.make_standard_montage('biosemi64')
filtered.set_channel_types({'EXG1': 'misc', 'EXG2': 'misc'})
filtered_montage = filtered.copy().set_montage(biosemi_montage, on_missing='warn')
filtered_montage.plot()

In [ ]:
ica = mne.preprocessing.ICA(n_components=20, random_state=0)
ica.fit(filtered_montage)
filtered_montage.load_data()
ica.plot_sources(filtered, show_scrollbars=True)

In [ ]:
ica.plot_components()

In [ ]:
# try the find_bads_eog() algorithm 
bad_idx, scores = ica.find_bads_eog(filtered_montage, ch_name=(['EXG3','EXG4','EXG5','EXG6']), threshold=2)
print(bad_idx)

In [ ]:
# ica.exclude = [0, 1, 6, 8, 9, 10] 
ica.exclude = bad_idx 
ica.apply(filtered_montage, exclude=ica.exclude).plot(events = events, event_id = event_id)

In [ ]:
stimulus_epochs = mne.Epochs(filtered_montage, events, event_id=event_id, tmin=-0.3, tmax=0.8, preload=True)
fig = stimulus_epochs.plot(events = events, event_id = event_id)

In [ ]:
# Downsample to 100 Hz
print('Original sampling rate:', stimulus_epochs.info['sfreq'], 'Hz')
epochs_resampled = stimulus_epochs.resample(256, npad="auto")
print('New sampling rate:', epochs_resampled.info['sfreq'], 'Hz')

In [ ]:
aver_epochs = epochs_resampled['sample'].average()

In [ ]:
fig1 = aver_epochs.plot()

In [ ]:
filtered_montage.plot()

In [ ]:
frontal_total = ['F1', 'F3', 'F5', 'F7', 'F2', 'F4', 'F6', 'F8', 'Fz']
parietal_total = ['P1', 'P3', 'P5', 'P7', 'P9', 'P2', 'P4', 'P6', 'P8', 'P10', 'Pz']

In [ ]:
evokeds = dict(average_frontal=aver_epochs)
picks = [f'F{n}' for n in range(1,9)]
mne.viz.plot_compare_evokeds(evokeds, picks=picks, combine='mean')

In [ ]:
evokeds = dict(average_parietal=aver_epochs)
picks = [f'P{n}' for n in range(1,12)]
mne.viz.plot_compare_evokeds(evokeds, picks=picks, combine='mean')